In [21]:
!pip install open_clip_torch

In [22]:
import os
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
import requests
import joblib
from torch.utils.data import DataLoader, Dataset

from sentence_transformers import SentenceTransformer
import open_clip

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)

# ------------------ Paths ------------------
TEST_CSV = "/content/drive/MyDrive/AMAZON25/test_cleaned1.csv"
MLP_PATH = "/content/drive/MyDrive/AMAZON25/best_multimodal_mlp_qwen06_4686.pth"
UNIT_ENCODER_PATH = "/content/drive/MyDrive/AMAZON25/unit_labelenc.joblib"
# os.makedirs(IMAGE_DIR, exist_ok=True)

# ------------------ Load Test CSV ------------------
test_df = pd.read_csv(TEST_CSV)
test_df["item_name"] = test_df["item_name"].fillna("").astype(str)
test_df["bullet_points"] = test_df.get("bullet_points", "").fillna("").astype(str)
test_df["value"] = pd.to_numeric(test_df.get("value", 0.0), errors="coerce").fillna(0.0)


# Load your trained LabelEncoder
le_unit = joblib.load(UNIT_ENCODER_PATH)

# Fill missing and convert to str
test_df["unit"] = test_df["unit"].fillna("unknown").astype(str)

# Replace any unseen labels with "unknown"
test_df["unit"] = test_df["unit"].apply(lambda x: x if x in le_unit.classes_ else "unknown")

# Now transform safely
unit_ids = le_unit.transform(test_df["unit"].tolist())


# TEST_EMB_DIR = "/content/drive/MyDrive/AMAZON25"


# ------------------ Load Qwen Text Embedding Model ------------------
# QWEN_SMALL_ID = "Qwen/Qwen3-Embedding-0.6B"
# print("Loading Qwen small embedding model...")
# qwen_model = SentenceTransformer(QWEN_SMALL_ID, device=DEVICE)
# NAME_DIM = qwen_model.get_sentence_embedding_dimension()
# BULLET_DIM = NAME_DIM
# print("Text embedding dim:", NAME_DIM)

# ------------------ Helper function: L2-normalize ------------------
# def l2_normalize(a, eps=1e-12):
#     norms = np.linalg.norm(a, axis=1, keepdims=True)
#     norms[norms < eps] = 1.0
#     return a / norms

# def encode_in_batches(model, texts, batch_size=128):
#     all_embs = []
#     for i in tqdm(range(0, len(texts), batch_size), desc="Encoding batches"):
#         batch = texts[i:i+batch_size]
#         emb = model.encode(batch, convert_to_numpy=True, show_progress_bar=False)
#         all_embs.append(emb)
#     return l2_normalize(np.vstack(all_embs))


#loading the saved ones
# ------------------ Load pre-saved item_name embeddings ------------------
name_emb_path = "/content/drive/MyDrive/AMAZON25/test_name_embs_qwen06.npy"
print(f"Loading item_name embeddings from {name_emb_path}...")
name_embs = np.load(name_emb_path)
print(f"Loaded item_name embeddings | shape={name_embs.shape}")

# ------------------ Load pre-saved bullet_points embeddings ------------------
bullet_emb_path = "/content/drive/MyDrive/AMAZON25/test_bullet_embs_qwen06.npy"
print(f"Loading bullet_points embeddings from {bullet_emb_path}...")
bullet_embs = np.load(bullet_emb_path)
print(f"Loaded bullet_points embeddings | shape={bullet_embs.shape}")


# ------------------ Generate and Save image embeddings ------------------
img_emb_path = "/content/drive/MyDrive/AMAZON25/test_image_embs_marqo.npz"
data = np.load(img_emb_path)
# Extract arrays from the file
img_embs = data["embeddings"]   # or the correct key name
img_ids = data["sample_ids"]           # or the correct key name

# ------------------ Load MLP Model ------------------


Device: cuda
Loading item_name embeddings from /content/drive/MyDrive/AMAZON25/test_name_embs_qwen06.npy...
Loaded item_name embeddings | shape=(75000, 1024)
Loading bullet_points embeddings from /content/drive/MyDrive/AMAZON25/test_bullet_embs_qwen06.npy...
Loaded bullet_points embeddings | shape=(75000, 1024)


In [23]:
# ------------------ Load MLP Model ------------------
import torch.nn as nn
import torch.nn.functional as F


PROJ_DIM = 256
BULLET_DIM = 1024
NAME_DIM = 1024
CAT_EMB_DIM = 32
num_unit_classes = len(le_unit.classes_)
IMAGE_DIM = img_embs.shape[1]   # <-- ADD THIS LINE

class MultiModalRegressor(nn.Module):
    def __init__(self, name_dim, bullet_dim, img_dim, num_unit_classes, proj_dim = 256, cat_emb_dim=32, dropout=0.3):
        super().__init__()
        # projections
        self.proj_name = nn.Sequential(nn.Linear(name_dim, proj_dim), nn.BatchNorm1d(proj_dim), nn.ReLU())
        self.proj_bullet = nn.Sequential(nn.Linear(bullet_dim, proj_dim), nn.BatchNorm1d(proj_dim), nn.ReLU())
        self.proj_img = nn.Sequential(nn.Linear(img_dim, proj_dim), nn.BatchNorm1d(proj_dim), nn.ReLU())

        # learnable unit embedding
        self.unit_emb = nn.Embedding(num_unit_classes, cat_emb_dim)
        nn.init.xavier_uniform_(self.unit_emb.weight)

        # numeric value projector
        self.value_proj = nn.Sequential(nn.Linear(1, 32), nn.ReLU())

        # gating layer (to weight modalities dynamically)
        self.gate = nn.Linear(proj_dim * 3, proj_dim * 3)

        # residual MLP block
        total_dim = proj_dim * 3 + cat_emb_dim + 32
        self.fc1 = nn.Linear(total_dim, total_dim)
        self.res_fc = nn.Linear(total_dim, total_dim)

        # head
        self.head = nn.Sequential(
            nn.Linear(total_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(dropout * 0.8),
            nn.Linear(128, 1)
        )

    def forward(self, name_emb, bullet_emb, img_emb, value, unit_ids):
        n = self.proj_name(name_emb)
        b = self.proj_bullet(bullet_emb)
        i = self.proj_img(img_emb)
        u = self.unit_emb(unit_ids)
        v = self.value_proj(value)

        # Gating
        gates = torch.sigmoid(self.gate(torch.cat([n, b, i], dim=1)))
        n = n * gates[:, :n.size(1)]
        b = b * gates[:, n.size(1):n.size(1)+b.size(1)]
        i = i * gates[:, -i.size(1):]

        # Combine
        x = torch.cat([n, b, i, u, v], dim=1)

        # Residual block
        x = x + self.res_fc(F.relu(self.fc1(x)))

        # Prediction head
        out = self.head(x)
        return out.squeeze(1)


mlp_model = MultiModalRegressor(NAME_DIM, BULLET_DIM, IMAGE_DIM, num_unit_classes, proj_dim=PROJ_DIM, cat_emb_dim=CAT_EMB_DIM, dropout=0.25).to(DEVICE)


mlp_model.load_state_dict(torch.load(MLP_PATH, map_location=DEVICE))
mlp_model.eval()

# ------------------ Prepare Tensors ------------------
value_tensor = torch.tensor(test_df["value"].values.reshape(-1,1), dtype=torch.float32).to(DEVICE)
unit_tensor = torch.tensor(unit_ids, dtype=torch.long).to(DEVICE)
name_tensor = torch.tensor(name_embs, dtype=torch.float32).to(DEVICE)
bullet_tensor = torch.tensor(bullet_embs, dtype=torch.float32).to(DEVICE)
img_tensor = torch.tensor(img_embs, dtype=torch.float32).to(DEVICE)

# ------------------ Batch Prediction ------------------
# ------------------ Batch Prediction with tqdm ------------------
from tqdm.auto import tqdm

batch_size = 1024
preds = []

print("🔮 Generating predictions...")
for i in tqdm(range(0, len(test_df), batch_size), desc="Predicting batches"):
    n = name_tensor[i:i+batch_size]
    b = bullet_tensor[i:i+batch_size]
    im = img_tensor[i:i+batch_size]
    v = value_tensor[i:i+batch_size]
    u = unit_tensor[i:i+batch_size]
    with torch.no_grad():
        batch_pred = mlp_model(n, b, im, v, u)
    preds.append(batch_pred.cpu().numpy())

# Concatenate all batch predictions
preds = np.concatenate(preds)

# If model trained on log1p(price)
prices = np.expm1(preds)

# ------------------ Save CSV with tqdm ------------------
print("💾 Creating submission CSV...")
submission_rows = []

for i in tqdm(range(len(test_df)), desc="Writing CSV rows"):
    submission_rows.append((test_df["sample_id"].iloc[i], prices[i]))

submission = pd.DataFrame(submission_rows, columns=["sample_id", "price"])

save_path = "/content/drive/MyDrive/AMAZON25/test_submissionXXXX.csv"
submission.to_csv(save_path, index=False)

print(f"✅ Submission CSV saved successfully: {save_path}")



🔮 Generating predictions...


Predicting batches:   0%|          | 0/74 [00:00<?, ?it/s]

💾 Creating submission CSV...


Writing CSV rows:   0%|          | 0/75000 [00:00<?, ?it/s]

✅ Submission CSV saved successfully: /content/drive/MyDrive/AMAZON25/test_submissionXXXX.csv
